In [3]:
import ScraperFC as sfc
import traceback
import pandas as pd
pd.options.display.max_columns = 50
from IPython.display import clear_output

scraper = sfc.Understat()
# error = False
# for yr in range(1988,2021):
#     for lg in ['EPL','La Liga','Bundesliga','Serie A','Ligue 1']:
try:
    lg_tbl = scraper.scrape_league_table(2019,'Serie A')
#     gk = scraper.scrape_adv_gk(2020,'La Liga',player=True)
#     passing = scraper.scrape_passing(2020,'EPL',normalize=True)
except:
    traceback.print_exc()
    scraper.close()
#     error = True
#     break
#     if error:
#         break
scraper.close()

In [4]:
lg_tbl

,Team,M,W,D,L,G,GA,PTS,xG,NPxG,xGA,NPxGA,NPxGD,PPDA,OPPDA,DC,ODC,xPTS
0,Juventus,38,28,6,4,70,30,90,64.53,58.44,35.03,31.99,26.45,9.77,13.03,290,158,70.93
1,Napoli,38,24,7,7,74,36,79,68.24,65.19,36.5,33.46,31.74,10.38,16.17,344,181,74.45
2,Atalanta,38,20,9,9,77,46,69,74.43,71.38,41.85,38.04,33.34,8.56,12.62,367,164,71.88
3,Inter,38,20,9,9,57,33,69,66.47,59.62,38.4,34.59,25.03,8.93,15.44,311,192,71.75
4,AC Milan,38,19,11,8,55,36,68,52.16,49.12,41.7,37.90,11.22,11.17,12.41,255,194,60.11
5,Roma,38,18,12,8,66,48,66,68.39,63.83,52.79,47.46,16.36,11.72,11.90,296,219,62.13
6,Torino,38,16,15,7,52,37,63,46.47,41.90,48.94,45.14,-3.23,7.11,8.34,219,215,49.84
7,Lazio,38,17,8,13,56,46,59,61.27,56.70,46.45,41.87,14.83,10.92,12.01,337,196,61.04
8,Sampdoria,38,15,8,15,60,51,53,53.19,44.06,54.27,48.05,-3.99,9.14,11.51,290,215,50.95
9,Bologna,38,11,11,16,48,56,44,48.1,44.29,51.06,48.01,-3.72,11.33,8.44,200,238,48.95
